<a href="https://colab.research.google.com/github/LaughBuddha/Deep-Learning---1/blob/master/AllCNN%20-%20finding%20a%20good%20learning%20schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
from torchvision import transforms, datasets
cifar_train = datasets.CIFAR10('./', download=True, train=True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])) 
cifar_val = datasets.CIFAR10('./', download=True, train=False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

0it [00:00, ?it/s]

170500096it [00:04, 40030495.75it/s]                               


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [0]:
cf_trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=128,
                                          shuffle=True, num_workers=2)
cf_testloader = torch.utils.data.DataLoader(cifar_val, batch_size=128,
                                         shuffle=False, num_workers=2)

In [0]:
class View(nn.Module):
    def __init__(self,o):
        super().__init__()
        self.o = o

    def forward(self,x):
        return x.view(-1, self.o)
    
class allcnn_t(nn.Module):
    def __init__(self, c1=96, c2= 192):
        super().__init__()
        d = 0.5

        def convbn(ci,co,ksz,s=1,pz=0):
            return nn.Sequential(
                nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
                nn.ReLU(True),
                nn.BatchNorm2d(co))

        self.m = nn.Sequential(
            nn.Dropout(0.2),
            convbn(3,c1,3,1,1),
            convbn(c1,c1,3,1,1),
            convbn(c1,c1,3,2,1),
            nn.Dropout(d),
            convbn(c1,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,2,1),
            nn.Dropout(d),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,10,1,1),
            nn.AvgPool2d(8),
            View(10))

    def forward(self, x):
        return self.m(x)
  

In [0]:
net = allcnn_t()

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

allcnn_t(
  (m): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Sequential(
      (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Dropout(p=0.5, inplace=False)
    (5): Sequential(
      (0): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

36
torch.Size([96, 3, 3, 3])


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9, weight_decay=0.001)

In [0]:
import numpy as np
loss_arr = np.empty((19550))
lr_arr = np.empty((19550))
error_arr = np.empty((19550))


loss_val_arr = np.empty((50))
lr_val_arr = np.empty((50))
error_val_arr = np.empty((50))

In [0]:
def calculate_lr(t, eta_max, T):
  T0 = T / 5
  if t <= (T0):
    eta = 10e-4 + (t / T0)*eta_max
  else:
    frac = (t - T0)/(T - T0)
    cos_val = (np.pi / 180)* 45 * frac
    eta = (eta_max * np.cos(cos_val)) + 10e-6
    
  return eta

In [0]:
epochs = 50
lr=0.00001
t = 0
correct = 0
total = 0

for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(cf_trainloader, 0):
    # get the inputs
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    # update the learning rate 
#     lr *= 1.0009
    lr = calculate_lr(t, eta_max = 0.05 / 10, T = 19550)
    lr_arr[t] = lr
    for g in optimizer.param_groups: 
      g['lr'] = lr
    
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, loss.item()))
    loss_arr[t] = loss.item()
    t += 1
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    error_arr[t] = total - correct / total
    
    # validation loss and error
  correct_val = 0
  total_val = 0
  with torch.no_grad():
    for data_val in cf_testloader:
        inputs_val, labels_val = data_val
        inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)

        outputs_val = net(inputs_val)
        loss_val = criterion(outputs_val, labels_val)
        _, predicted_val = torch.max(outputs_val.data, 1)
        total_val += labels_val.size(0)
        correct_val += (predicted_val == labels_val).sum().item()

  print('Validation loss after %d weight updates : %f ' % 
        ( t , loss_val.item()))
  loss_val_arr[epoch] = loss_val.item()
  lr_val_arr[epoch] = t

  print('Validation error after %d weight updates : %f %%' % 
        ( t , (total_val - correct_val) / total_val))
  error_val_arr[epoch] = total_val - correct_val / total_val

[1,     1] loss: 2.306
[1,     2] loss: 2.337
[1,     3] loss: 2.321
[1,     4] loss: 2.330
[1,     5] loss: 2.322
[1,     6] loss: 2.273
[1,     7] loss: 2.310
[1,     8] loss: 2.288
[1,     9] loss: 2.295
[1,    10] loss: 2.311
[1,    11] loss: 2.278
[1,    12] loss: 2.291
[1,    13] loss: 2.242
[1,    14] loss: 2.274
[1,    15] loss: 2.223
[1,    16] loss: 2.233
[1,    17] loss: 2.237
[1,    18] loss: 2.209
[1,    19] loss: 2.252
[1,    20] loss: 2.212
[1,    21] loss: 2.188
[1,    22] loss: 2.226
[1,    23] loss: 2.198
[1,    24] loss: 2.177
[1,    25] loss: 2.150
[1,    26] loss: 2.179
[1,    27] loss: 2.236
[1,    28] loss: 2.182
[1,    29] loss: 2.151
[1,    30] loss: 2.129
[1,    31] loss: 2.113
[1,    32] loss: 2.124
[1,    33] loss: 2.142
[1,    34] loss: 2.192
[1,    35] loss: 2.171
[1,    36] loss: 2.061
[1,    37] loss: 2.125
[1,    38] loss: 2.149
[1,    39] loss: 2.067
[1,    40] loss: 2.144
[1,    41] loss: 2.041
[1,    42] loss: 2.127
[1,    43] loss: 2.082
[1,    44] 

IndexError: ignored

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(12, 8), dpi=80,)
ax.set(xscale="log")

# plt.plot(lr_arr, loss_arr)
# plt.plot(loss_arr)
# plt.plot(error_arr)
# plt.plot(loss_val_arr)
# plt.plot(error_val_arr)
